# Как нейросетки смотрят на мир 

In [1]:
import tensorflow as tf
print(tf.__version__)

2.0.0


In [2]:
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# Подгружаем предобученную на imagenet VGG19
model = tf.keras.applications.VGG19(include_top=False, 
                                        weights='imagenet')

model.trainable = False # Запрещаем ей обучаться 

80142336/80134624 [==============================] - 53s 1us/step


In [4]:
model.summary()

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, None, 3)]   0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

In [13]:
layer_name = 'block3_conv1'

In [15]:
from tensorflow.keras import Model

def vgg_layers(layer_name):

    # загрузили модель из зоопарка
    model = tf.keras.applications.VGG19(include_top=False, weights='imagenet')
    model.trainable = False
    
    # вытащили те слои, которые пойдут на выход
    output = model.get_layer(layer_name).output
    
    # собрали модель: на вход изображение, на выход слои
    model = Model([model.input], output)
    return model

In [16]:
model = vgg_layers(layer_name)

In [24]:
def loss(out):
    return -1*tf.reduce_mean(out)

In [44]:
input_img_data = np.random.random((1, 150, 150, 3), type=float32) * 20 + 128.

plt.imshow(np.squeeze(input_img_data,0))

NameError: name 'float32' is not defined

In [26]:
model.predict(input_img_data).shape

(1, 37, 37, 256)

In [27]:
loss(model.predict(input_img_data).shape)

<tf.Tensor: id=1711, shape=(), dtype=int32, numpy=-82>

In [36]:
class StyleContentModel(tf.keras.models.Model):
    
    def __init__(self, layer_name):
        super(StyleContentModel, self).__init__()
        
        # модель
        self.model =  vgg_layers(layer_name)
        self.model.trainable = False

        
    def call(self, image):

        # в вектор outputs запишутся все наши активации
        output = self.model(image)
        
        return output

Задаём процедуру обучения!

In [28]:
image = tf.Variable(input_img_data)

def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [29]:
opt = tf.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

In [37]:
extractor = StyleContentModel(layer_name)

In [38]:
@tf.function()
def train_step(image):
    
    # открыли окружение для проброски градиентов
    with tf.GradientTape() as tape:
        
        # строим прогнозы 
        filter_value = extractor(image)
        
        # считаем потери
        loss_value = loss(filter_value)
    
    # находим градиент
    grad = tape.gradient(loss_value, image)
    
    # делаем шаг градиентного спуска 
    opt.apply_gradients([(grad, image)])
    
    # следим, чтобы картинка не вышла за отрезок [0;1 ]
    image.assign(clip_0_1(image))

In [39]:
train_step(image)

TypeError: in converted code:

    <ipython-input-38-27782cb86c75>:8 train_step  *
        filter_value = extractor(image)
    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    <ipython-input-36-a87078096231>:14 call  *
        output = self.model(image)
    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs = call_fn(cast_inputs, *args, **kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:708 call
        convert_kwargs_to_constants=base_layer_utils.call_context().saving)
    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/network.py:860 _run_internal_graph
        output_tensors = layer(computed_tensors, **kwargs)
    /usr/local/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:847 __call__
        outputs =